<a href="https://colab.research.google.com/github/heejinyoon/SeismicBump-/blob/main/septis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import io
import requests

from plotly.offline import iplot
import plotly.graph_objs as go
from plotly import tools
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import seaborn as sns


from datetime import tzinfo, timedelta, datetime, date

import os
from os.path import join, getsize
from pathlib import Path



In [ ]:
# import SKLearn libs
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV, train_test_split, StratifiedKFold, cross_validate, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier


Early Prediction of Sepsis from Clinical Data -- the PhysioNet Computing in Cardiology Challenge 2019
Matthew Reyna  ,  Chris Josef  ,  Russell Jeter  ,  Supreeth Shashikumar  ,  Benjamin Moody  ,  M. Brandon Westover  ,  Ashish Sharma  ,  Shamim Nemati  ,  Gari Clifford 

Published: Aug. 5, 2019. Version: 1.0.0
https://physionet.org/content/challenge-2019/1.0.0/

Creative Commons Attribution 4.0 International Public License

https://archive.physionet.org/users/shared/challenge-2019/

In [ ]:
training_setAzip = "https://archive.physionet.org/users/shared/challenge-2019/training_setA.zip"
training_setBzip = "https://archive.physionet.org/users/shared/challenge-2019/training_setB.zip"

!wget {training_setAzip} 
!wget {training_setBzip} 


--2020-09-06 19:22:01--  https://archive.physionet.org/users/shared/challenge-2019/training_setA.zip
Resolving archive.physionet.org (archive.physionet.org)... 128.30.30.88
Connecting to archive.physionet.org (archive.physionet.org)|128.30.30.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22991842 (22M) [application/zip]
Saving to: ‘training_setA.zip.1’

training_setA.zip.1 100%[===================>]  21.93M  24.4MB/s    in 0.9s    

2020-09-06 19:22:02 (24.4 MB/s) - ‘training_setA.zip.1’ saved [22991842/22991842]

--2020-09-06 19:22:02--  https://archive.physionet.org/users/shared/challenge-2019/training_setB.zip
Resolving archive.physionet.org (archive.physionet.org)... 128.30.30.88
Connecting to archive.physionet.org (archive.physionet.org)|128.30.30.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20000482 (19M) [application/zip]
Saving to: ‘training_setB.zip.1’

training_setB.zip.1 100%[===================>]  19.07M  23.7MB

In [ ]:
from pathlib import Path

# set up local directory
base_dir = Path("septis")
base_dir.mkdir(exist_ok=True)
base_dirA = Path("septis/setA")
base_dirA.mkdir(exist_ok=True)
pathA = "/content/" + str( base_dirA)

base_dirB = Path("septis/setB")
base_dirB.mkdir(exist_ok=True)
pathB = "/content/" + str( base_dirB)

In [ ]:
!unzip "/content/training_setA.zip" -d {pathA}


Archive:  /content/training_setA.zip
replace /content/septis/setA/training/p000001.psv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
path = "/content/septis"

In [ ]:
!unzip "/content/training_setB.zip" -d {pathB}

# Have a look at 1 file

In [ ]:
fullpath = "/content/septis/setB/training_setB/p104592.psv"
patientdata = pd.read_csv(fullpath, sep='|', na_values='NaN')
patientdata.columns

In [ ]:
patientdata.loc[:, ['ICULOS', 'SepsisLabel']]

# Load all files raw data into 1 Pandas frame

In [ ]:
# Load the 1st (2*patientfilesmax) patient files (50 with no septis and 50 with septis) to understand data better
noseptispatientfiles = []
noseptispatientcount = 0
septispatientfiles = []
septispatientcount = 0
noseptis_patientfilesmax = 200
septis_patientfilesmax = 800
patientids = []
patients = []

for dirname, dirs, filenames in os.walk('/content/septis/'):
    print(dirname, "consumes", end=" ")
    #print(sum(getsize(join(dirname, name)) for name in filenames), end=" ")
    print("bytes in", len(filenames), "non-directory files")
    for filename in filenames[0:10000]:
        fullpath = os.path.join(dirname, filename)
        patientdata = pd.read_csv(fullpath, sep='|', na_values='NaN')
        if ((patientdata['SepsisLabel'].max() == 0) & (noseptispatientcount < noseptis_patientfilesmax)):
          noseptispatientcount = noseptispatientcount + 1
          patients.append(patientdata)
          patientids.append(filename)
          #print("No septis: " + fullpath)
        elif ((patientdata['SepsisLabel'].max() == 1) & (septispatientcount < septis_patientfilesmax)):
          septispatientcount = septispatientcount + 1
          patients.append(patientdata)
          patientids.append(filename)
          #print("Septis: " + fullpath)
df = pd.concat(patients, keys=patientids)


In [ ]:
print("septispatientcount: " + str(septispatientcount) + " noseptispatientcount: " + str(noseptispatientcount))


In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df

In [ ]:
df_description = df.describe().T
df_description

In [ ]:
df.describe(include=['object']).T

In [ ]:
# Find which columns have a lot of Null values
df.isnull().sum()

In [ ]:
plt.hist(df['Temp'], bins=20)
plt.title("Histogram")
plt.show()



In [ ]:
sns.countplot(df['SepsisLabel'])

In [ ]:
df['Temp']

# Luciana

In [ ]:
#Handling Outliers(data samples distant from all other observations)

outliers=[]
def outliers_noplotdetection(df):
  threshold=3
  mean_score = np.mean(df)
  std_score = np.std(df)

  for df_sample in df:
    z_score = (df_sample - mean_score)/std_score
    if np.abs(z_score)>threshold:
      outliers.append(df_sample)
  return outliers



In [ ]:
mean_score = np.mean(df)
mean_score

In [ ]:
#outliers = outliers_noplotdetection(df)

# Catherine section

## Vital Signs

In [ ]:
# Analyze Vital Signs measuremnts and correlation with Sepsis
label = 'SepsisLabel'
col_list_categorical = ['level_0'] # patient code
col_list_vital_signs = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2']
df[col_list_vital_signs].describe()

In [ ]:
# ignore EtCO2 as very value
col_list_for_corr = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'level_1', 'SepsisLabel']
df_corr = df[col_list_for_corr].corr()

In [ ]:
plt.figure(figsize=[8, 8])
sns.heatmap(data=df_corr, vmin=-1, vmax=1, cmap='gist_earth_r', annot=True, square=True, linewidths=1)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

### Very small correlation. 
> Level_1 (time since entry to ICU, same as ICULOS) has slight correlation as might be expected (patient might get worse as time passes)

> ignore EtCO2 as no value


## Demographics

In [ ]:
col_list_demographics = ['Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS']
df[col_list_demographics].describe()

In [ ]:
col_list_for_corr = ['Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS', 'SepsisLabel']
df_corr = df[col_list_for_corr].corr()

plt.figure(figsize=[8, 8])
sns.heatmap(data=df_corr, vmin=-1, vmax=1, cmap='gist_earth_r', annot=True, square=True, linewidths=1)
plt.xticks(rotation=90)
plt.yticks(rotation=0)


### Very small correlation again. 
> Level_1 (time since entry to ICU, same as ICULOS) has slight correlation as might be expected (patient might get worse as time passes)



## Laboratory measures 

In [ ]:
col_list_Laboratory = ['BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2',
       'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine',
       'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium', 'Phosphate',
       'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets']
df[col_list_Laboratory].describe().T.sort_values(by='count', ascending=False)

In [ ]:
col_list_for_corr = ['BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2',
       'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate',
       'Potassium', 'Bilirubin_total', 'Hct', 'Hgb', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets', 'SepsisLabel']  
df_corr = df[col_list_for_corr].corr()

plt.figure(figsize=[16, 16])
sns.heatmap(data=df_corr, vmin=-1, vmax=1, cmap='gist_earth_r', annot=True, square=True, linewidths=1)
plt.xticks(rotation=90)
plt.yticks(rotation=0)

### Very sparse date, and small correlation again.

> Ignore 'Bilirubin_direct', 'TroponinI', as very few values

> Hgb and Hct redundant?


> white blood cells (WBCs) correlation makes sense , as reaction to infection.





## Forward fill measures per patient

In [ ]:
col_to_fill = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 
       'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN',
       'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
       'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium',
       'Bilirubin_total', 'TroponinI', 'Hct', 'PTT', 'WBC',
       'Fibrinogen', 'Platelets']

col_filled = ['Age', 'Gender', 'HospAdmTime', 'ICULOS']

col_ignore = ['level_0', 'level_1', 'Unit1', 'Unit2', 'Hgb', 'EtCO2'] 
col_delta = ['HR_delta', 'Temp_delta', 'O2Sat_delta']

In [ ]:
# Returns patientdata with filled values
def fillMissingValues(filepath):
  patientdata = pd.read_csv(filepath, sep='|', na_values='NaN')

  # mark where there where Null values
  #cols_with_missing = [col for col in col_to_fill 
  #                     if patientdata[col].isnull().any()]

  for col in col_to_fill:
    patientdata[col + '_was_missing'] = patientdata[col].isnull()

  # Forward fill available values
  patientdata.fillna(method='ffill', inplace=True)

  # Delta values: diff with previous records
  patientdata["HR_delta"] = patientdata["HR"].diff(periods=1)
  patientdata["Temp_delta"] = patientdata["Temp"].diff(periods=1)
  patientdata["O2Sat_delta"] = patientdata["O2Sat"].diff(periods=1)

  # Fill remaining values with mean values from original data
  for col in col_to_fill:
    values = {col: df_description['mean'][col]}
    patientdata.fillna(value=values, inplace=True)  

  for col in col_delta:
    values = {col: 0}
    patientdata.fillna(value=values, inplace=True) 

  return patientdata


In [ ]:
fullpath = "/content/septis/setB/training_setB/p104592.psv"
dt = fillMissingValues(fullpath)
dt

In [ ]:
 # Load the 1st 100 patient files (50 with no septis and 50 with septis) to understand data better
noseptispatientfiles = []
noseptispatientcount = 0
septispatientfiles = []
septispatientcount = 0
noseptis_patientfilesmax = 100
septis_patientfilesmax = 800
patientids = []
patients = []

no_septis_tests_files=[]
septis_tests_files=[]

for dirname, dirs, filenames in os.walk('/content/septis/'):
    print(dirname, "consumes", end=" ")
    #print(sum(getsize(join(dirname, name)) for name in filenames), end=" ")
    print("bytes in", len(filenames), "non-directory files")
    for filename in filenames[0:6000]:
        fullpath = os.path.join(dirname, filename)
        patientdata = fillMissingValues(fullpath)
        if (patientdata['SepsisLabel'].max() == 0):
          if (noseptispatientcount < noseptis_patientfilesmax):
            noseptispatientcount = noseptispatientcount + 1
            patients.append(patientdata)
            patientids.append(filename)
            #print("No septis: " + fullpath)
          else:
            no_septis_tests_files.append(fullpath)
        elif (patientdata['SepsisLabel'].max() == 1):
          if (septispatientcount < septis_patientfilesmax):
            septispatientcount = septispatientcount + 1
            patients.append(patientdata)
            patientids.append(filename)
            #print("Septis: " + fullpath)
          else:
            septis_tests_files.append(fullpath)

df_ffill = pd.concat(patients, keys=patientids)

print("septispatientcount: " + str(septispatientcount) + " noseptispatientcount: " + str(noseptispatientcount))

df_ffill.reset_index(inplace=True)


In [ ]:
print("septispatient test files: " + str(len(septis_tests_files)) + 
      " no septispatient test files: " + str(len(no_septis_tests_files)))


In [ ]:
cols_with_missing = [o for o in df_ffill.columns if o.endswith('_was_missing')]
cols_with_missing

In [ ]:
# Compare number of Null Values of Forward-filled data with Raw Data

df_ffill_description = df_ffill.describe().T.sort_values(by='count', ascending=False)
ffilnull = df_ffill.isnull().sum()
dfnull = df.isnull().sum()
nulldf = pd.DataFrame({'rawdf': dfnull, 'ffilldf': ffilnull})
nulldf.head(40)

In [ ]:
nulldf.tail(40)

In [ ]:
df_ffill.columns

In [ ]:
# Check no colun missed!
print("Colums num: " + str( len(col_to_fill) + len(col_filled) + len(col_delta)  + len(col_ignore) + len(cols_with_missing) + 1) + " : " + str(len(df_ffill.columns)))


## Set-up X and y datests for learning

In [ ]:
Xcols = (col_to_fill + col_filled + col_delta + cols_with_missing)
X = (df_ffill.loc[:, Xcols]).copy()

y = df_ffill.SepsisLabel

In [ ]:
# spliting data set in ratio 8:2 for training and testing 
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.2, random_state = 0)

In [ ]:
print('Training Set:')
print('Features', train_X.shape, '\nLabel', train_y.shape)
print('Testing Set:')
print('Features', val_X.shape, '\nLabel', val_y.shape)

In [ ]:
# fit and transform data 
sc = StandardScaler()
sc.fit_transform(train_X)
sc.transform(val_X)


## Try Random Forest Classifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=800)
rfc.fit(train_X, train_y)
pred_rfc = rfc.predict(val_X)

In [ ]:
print(classification_report(val_y, pred_rfc))




---
      -                    Actual Positive      Actual Negative
      Predictive Positive  True Positive        False Positive
      Predicted Negative.  False Negative.      True Negative


In [ ]:
print(confusion_matrix(val_y, pred_rfc))

In [ ]:
from pprint import pprint
# Look at parameters used by our current forest
print('Parameters currently in use:\n')
pprint(rfc.get_params())


In [ ]:
train_X.shape

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

## TODO: finetune Random Forest Classifier

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rfc = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, 
#                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
#rfc_random.fit(train_X, train_y)

#rfc_random.best_params_

In [ ]:
#rfc_random.best_params_

# VERY LONG optimisation , comment out , results here:
{'bootstrap': False,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1100}



```
Fitting 3 folds for each of 100 candidates, totalling 300 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 61.3min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 247.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 509.2min finished
RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators=100,
                                                    n_jobs...
                                                    warm_start=False),
                   iid='deprecated', n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 32, 55, 77, 100,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 650, 1100, 1550,
                                                         2000]},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

# This is formatted as code
```


In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Use the random grid to search for best hyperparameters
# First create the base model to tune
#rfc = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
#rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, 
#                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
#rfc_random.fit(train_X, train_y)


In [ ]:
best_rfc = RandomForestClassifier(bootstrap = False, max_depth = None, max_features = 'sqrt',
                                  min_samples_leaf = 1, min_samples_split = 2, n_estimators = 1100)
best_rfc.fit(train_X, train_y)
best_pred_rfc = best_rfc.predict(val_X)

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1100}

 

In [ ]:
print(classification_report(val_y, best_pred_rfc))

print(confusion_matrix(val_y, best_pred_rfc))

In [ ]:

8445 / (8445 + 310)

Non fine-tuned RFC

```
[[8243   83]
 [ 331 1223]]
```

With Null columns

```
[[8457   76]
 [ 450 1126]]
```

Fined-tuned RFC


```
[[8241   85]
 [ 232 1322]]
```


Fined-tuned RFC with NULL columns
```
[[8445   88]
 [ 310 1266]]
```







In [ ]:
#RandomForestClassifier(bootstrap = False, max_depth = None, max_features = 'sqrt',
#                                  min_samples_leaf = 1, min_samples_split = 2, n_estimators = 1100)
#best_rfc.fit(train_X, train_y)
#best_pred_rfc = best_rfc.predict(val_X)

In [ ]:
def rf(xs, y, n_estimators=100, 
       max_features='sqrt', min_samples_leaf=5, **kwargs):
    return RandomForestClassifier(n_jobs=-1, n_estimators=n_estimators,
        max_features=max_features,
        min_samples_leaf=min_samples_leaf, oob_score=True).fit(xs, y)

In [ ]:
m = rf(train_X, train_y);
m_pred = m.predict(val_X)
cm = confusion_matrix(val_y, m_pred)
print(cm)
cm[1, 1] / (cm[1, 1] + cm[1, 0])



In [ ]:
preds = np.stack([t.predict(val_X) for t in m.estimators_])

In [ ]:
def myrecall(y_true, y_pred):
  class_pred = [1 if t >=0.5 else 0 for t in y_pred]
  cm = confusion_matrix(y_true, class_pred)
  #print(cm)
  return (cm[1, 1] / (cm[1, 1] + cm[1, 0]))



In [ ]:
plt.plot([myrecall(val_y, preds[:i+1].mean(0)) for i in range(100)]);

In [ ]:
m = rf(train_X, train_y, max_features="auto")
preds = np.stack([t.predict(val_X) for t in m.estimators_])

In [ ]:
plt.plot([myrecall(val_y, preds[:i+1].mean(0)) for i in range(100)]);

## Pretty good results :=) 
> now, what are the most important model features in this model?

In [ ]:
importances = best_rfc.feature_importances_
std = np.std([tree.feature_importances_ for tree in best_rfc.estimators_], axis=0)
indices = np.argsort(importances)[::-1]
 
# Print the feature ranking
print("Feature ranking:")
 
for f in range(train_X.shape[1]):
    print("%d. feature %d: %s (%f)" % (f + 1, indices[f], train_X.columns[f], importances[indices[f]]))
 


## HR Heart Rate is the Top feature in that model

# Heejin Section

# Missing value visualization 

In [ ]:
import missingno as msno 
lab_df = df[col_list_Laboratory]
vital_df = df[col_list_vital_signs]
demograph_df = df[col_list_demographics]

In [ ]:
np.mean((lab_df.isnull().sum() / lab_df.shape[0]))

In [ ]:
np.mean((vital_df.isnull().sum() / vital_df.shape[0]))

In [ ]:
np.mean((demograph_df.isnull().sum() / demograph_df.shape[0]))

In [ ]:
msno.matrix(lab_df)

In [ ]:
msno.matrix(vital_df)

In [ ]:
msno.matrix(demograph_df)

In [ ]:
missingDataDf = df.columns[df.isnull().any()].tolist()
msno.bar(df[missingDataDf], color="Red", log=False, figsize=(35,13))

In [ ]:
msno.heatmap(df[missingDataDf], figsize=(25,25))

# Data visualization

In [ ]:
df.level_0.value_counts()

In [ ]:
#subplots
plt.figure(figsize = (10,8))

#KDE plot - smoothed histograms showing distribution of a variable for sepsis 
#           patients/non-sepsis patients 
patientsdf = df.groupby('level_0').agg(
    min_ICULOS = ('ICULOS', min),
    max_ICULOS = ('ICULOS', max),
    min_SepsisLabel = ('SepsisLabel', min),
    max_SepsisLabel = ('SepsisLabel', max),
    min_HeartRate = ('HR', min),
    max_HeartRate = ('HR', max),
    min_Temperature = ('Temp', min),
    max_Temperature = ('Temp', max),
    Age = ('Age', min),
    Gender = ('Gender', min)
)
# check there are 100 patients who ended up having sepsis at some point and 100 who didn't
#patientsdf.max_SepsisLabel.value_counts()

sns.kdeplot(patientsdf.loc[patientsdf['max_SepsisLabel'] == 0]['Age'], label = 'Non-sepsis Patient')
sns.kdeplot(patientsdf.loc[patientsdf['max_SepsisLabel'] == 1]['Age'], label = 'Sepsis Patient')

#Labeling of plot 
plt.xlabel('Age (years)'); plt.ylabel('Sepsis'); plt.title('Distribution of Ages')


In [ ]:
patientsdf.columns

In [ ]:
plt.hist(patientsdf['max_ICULOS'], bins=20)
plt.title("Max Number of Hours in ICU Histogram")

plt.show()

In [ ]:
f, ax = plt.subplots(2,2, figsize=(15,10))
sns.countplot(x = patientsdf["max_SepsisLabel"], palette = 'Blues', ax=ax[0,0])
ax[0,0].set_title('Count of patients who develop sepsis (1) vs. don\'t (0) ')

sns.set(style='whitegrid')
sns.countplot(x = df["SepsisLabel"], palette = 'Blues', ax=ax[0,1])
ax[0,1].set_title('Count of total time slots with sepsis ')

ax[1,0].hist(patientsdf.loc[patientsdf['max_SepsisLabel'] == 0]['max_ICULOS'], bins=20, )
ax[1,0].set_title("Distribution of max number of hours in ICU for patients who don't develop Sespis")

ax[1,1].hist(patientsdf.loc[patientsdf['max_SepsisLabel'] == 1]['max_ICULOS'], bins=20, )
ax[1,1].set_title("Distribution of max number of hours in ICU for patients who develop Sespis")


plt.show()

In [ ]:
# Compare temperature between sepsis patient and non sepsis patient 
fig = go.Figure()
fig.add_trace(go.Box(y=patientsdf.loc[patientsdf['max_SepsisLabel'] == 1]['max_Temperature'], name='Sepsis patient Temperature'))

fig.add_trace(go.Box(y=patientsdf.loc[patientsdf['max_SepsisLabel'] == 0]['max_Temperature'], name='Non Sepsis patient Temperature'))

fig.show()

In [ ]:
# Compare heart rate between sepsis patient and non sepsis patient 
fig = go.Figure()
fig.add_trace(go.Box(y=patientsdf.loc[patientsdf['max_SepsisLabel'] == 1]['max_HeartRate'], name='Sepsis patient Heart Rate'))

fig.add_trace(go.Box(y=patientsdf.loc[patientsdf['max_SepsisLabel'] == 0]['max_HeartRate'], name='Non Sepsis patient Heart Rate'))

fig.show()


# Logistic Regression Model

In [ ]:
# Defining pipeline 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer

category_LR = train_X.dtypes == object

cat_pipeline = make_pipeline(OneHotEncoder(handle_unknown = "ignore"))
num_pipeline = make_pipeline(SimpleImputer(strategy = 'median'), StandardScaler())

preprocessor = make_column_transformer((num_pipeline, ~category_LR), (cat_pipeline, category_LR))

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# Training the algorithm  
log_pip_scale = make_pipeline(preprocessor, LogisticRegression(solver = 'lbfgs', n_jobs = -1, C=0.01, random_state = 0, max_iter= 1000) )

logModel = log_pip_scale.fit(train_X, train_y)

In [ ]:
# Predict the test set result 
y_pred = logModel.predict(val_X)
acc = accuracy_score(val_y, y_pred)

# Confusion Matrix 
conf_matrix = confusion_matrix(val_y, y_pred)

# Print result 
print('Prediction value counts:')
print(pd.Series(y_pred).value_counts())
print('\nAccuracy on the training data is {0:f}'.format(acc))
conf_matrix

In [ ]:
print(classification_report(val_y, y_pred))

In [ ]:
# visualizing confusion Matrix using heatmap 
class_name = [0,1]
fig, ax = plt.subplots()
tick_mark = np.arange(len(class_name))
plt.xticks(tick_mark, class_name)
plt.yticks(tick_mark, class_name)

sns.heatmap(pd.DataFrame(conf_matrix), annot=True, cmap="YlGnBu", fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('confusion matrix', y = 1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print("Accuracy:", accuracy_score(val_y, y_pred))
print("Precision:", precision_score(val_y, y_pred))
print("Recall:", recall_score(val_y, y_pred))

# Decision Tree Model 

In [ ]:
# from sklearn.compose import ColumnTransformer
# # Training the algorithm  
# dtm_pip_scale = make_pipeline(preprocessor, tree.DecisionTreeClassifier() )

# dtModel = dtm_pip_scale.fit(train_X, train_y)

In [ ]:
# # Predict the test set result 
# y_pred = dtModel.predict(val_X)
# acc = accuracy_score(val_y, y_pred)

# # Confusion Matrix 
# conf_matrix = confusion_matrix(val_y, y_pred)

# # Print result 
# print('Prediction value counts:')
# print(pd.Series(y_pred).value_counts())
# print('\nAccuracy on the training data is {0:f}'.format(acc))
# conf_matrix

In [ ]:
# # get data using pipeline 
# cat_names = preprocessor.fit(train_X).named_transformers_['pipeline-2'].named_steps['onehotencoder']

# feature_list = list(cat_names.get_feature_names())
# feature_list.extend(['ICULOS','HospAdmTime'])

# import graphviz 
# dot_data = tree.export_graphviz(tree.DecisionTreeClassifier(max_leaf_nodes = 15).fit(preprocessor.fit_transform(train_X), train_y),
#                                 out_file=none, feature_names = feature_list, class_name = ['normal','sepsis'],
#                                 rounded = True, proportion = False, precision = 2)
# graph = graphviz.Source(dot_data)
# graph.render("iris")



In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_tree_classifier = DecisionTreeClassifier()
DTree = decision_tree_classifier.fit(train_X,train_y)
predictedDT = decision_tree_classifier.predict(val_X)
before_fs=metrics.accuracy_score(predictedDT,val_y)*100
print("Accuracy using DecisionTreeClassifier:",before_fs)